# Python爬蟲

### 1. 進入到搜尋列表

In [6]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# 關閉 Alert
options = Options()
options.add_argument("--disable-notifications")

# 開啟網頁
driver = webdriver.Chrome(
    ChromeDriverManager().install(), chrome_options=options)
driver.get("https://twpat3.tipo.gov.tw/twpatc/twpatkm?@@0.6886565575531223")
driver.implicitly_wait(6)# 等待最多6秒

driver.find_element_by_xpath('//*[@id="datepicker_Start"]/input[2]').send_keys('20000101')
driver.find_element_by_xpath('//*[@id="datepicker_End"]/input[2]').send_keys('20211231')

    
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[4]/td/span[1]/input').click()# 點選查詢
driver.implicitly_wait(10)# 等待最多10秒


record = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/div/table/tbody/tr/td[1]/select')# 選筆數格子
select = Select(record)
select.select_by_visible_text("100")# 選筆數
driver.implicitly_wait(10)# 等待最多10秒

# 選文字式顯示
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/ul[2]/li[4]/a/img').click()
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/ul[2]/li[4]/ul/li[3]/input').click()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Get LATEST driver version for 96.0.4664
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Neil\.wdm\drivers\chromedriver\win32\96.0.4664.45]
C:\Users\Neil\AppData\Local\Temp/ipykernel_16152/1125642898.py:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


#### 問題:寫成函式呼叫?

## 2. 擷取網頁資訊

In [7]:
# 計數器歸0，重新輸入資料
count = 0

In [8]:
from collections import defaultdict
import pandas as pd

for i in range(1):# 抓多少頁
    
    list1 = []
    list2 = []
    list3 = []
    list4 = []
    list5 = []
    list6 = []
    
    driver.implicitly_wait(10)# 等待最多10秒
    
    # 抓取公開公告號號
    lication_nums = driver.find_elements_by_class_name('sumtd2012_603')
    for lication_num in lication_nums:
        list1.append(lication_num.text)

    # 抓取公開公告日
    release_dates = driver.find_elements_by_class_name('sumtd2025_603')
    for release_date in release_dates:
        list2.append(release_date.text)
    
    # 抓取申請日
    application_dates = driver.find_elements_by_class_name('sumtd2021_603')
    for application_date in application_dates:
        list3.append(application_date.text)

    # 抓取標題
    titles = driver.find_elements_by_class_name('sumtd2030_603')
    for title in titles:
        list4.append(title.text)
        
    # 抓取摘要
    summaries = driver.find_elements_by_class_name('sumtd2095_603')
    for summary in summaries:
        list5.append(summary.text)
        
    # 抓取專利範圍
    constructions = driver.find_elements_by_class_name('sumtd2096_603')
    for construction in constructions:
        list6.append(construction.text)

    # 資料轉成dict
    length = len(list1)    
    dict_from_list = {
        "lication_nums": [list1[i] for i in range(length)],
        "release_dates": [list2[i] for i in range(length)],
        "application_dates": [list3[i] for i in range(length)],
        "titles": [list4[i] for i in range(length)],
        "summaries": [list5[i] for i in range(length)],
        "constructions": [list6[i] for i in range(length)]
    }
    df = pd.DataFrame(dict_from_list)
    
    # 資料存成csv格式
    if count == 0:
        df.to_csv("oringinal_data.csv", index = False)
        count = count + 1
    else:
        df.to_csv("oringinal_data.csv", mode = 'a', header = False, index = False)

    
    driver.find_element_by_name('_IMG_次頁').click()# 點選下一頁

# 關driver
driver.quit()

#### 問題:需解決.csv亂碼?字典排列形式有更好的?設計避免重複抓取的判斷?儲存在更好的SQL?

## 3.資料匯入程式

In [14]:
import pandas as pd

# csv輸入至oringinal_csv 
df = pd.read_csv("oringinal_data.csv", encoding="utf-8-sig")
df.head(3)

,lication_nums,release_dates,application_dates,titles,summaries,constructions
0,M620847,2021/12/01,2021/10/18,冷水循環系統,本創作提供一種冷水循環系統，其將一冷卻裝置組包含之一第一冷卻裝置以一第一管路以及一第二管路連...,1.一種冷水循環系統，其包含： 一冷卻裝置組，其包含一第一冷卻裝置以及一第二冷卻裝置，該第一...
1,M620834,2021/12/01,2021/09/29,線路板與發光裝置\nCIRCUIT BOARD AND LIGHT-EMITTING APP...,一種線路板，包括基材、線路層、第一阻焊層以及第二阻焊層。線路層位於基材上。線路層包括第一接墊...,1.一種線路板，包括： 基材； 線路層，位於所述基材上，且所述線路層包括第一接墊及第二接墊；...
2,M620830,2021/12/01,2021/09/22,噴灑式再加熱型蒸汽反應裝置及使用其的發電設備\nSPRAYING AND RE-HEATIN...,一種噴灑式再加熱型蒸汽反應裝置包含：一保溫爐；一蒸汽反應爐，設置於保溫爐中；一再加熱管路，將...,1.一種噴灑式再加熱型蒸汽反應裝置，包含： 一保溫爐； 一蒸汽反應爐，設置於該保溫爐中； 一...


## 4.中文分詞

In [15]:
import re
from bs4 import BeautifulSoup

# 去除\n
def clean_post_cont(x):
    soup = BeautifulSoup(x)
    result = soup.get_text()
    return re.sub(r"\r|\n|\t", "", result)

df["titles"] = df["titles"].map(clean_post_cont)
df["summaries"] = df["summaries"].map(clean_post_cont)
df["constructions"] = df["constructions"].map(clean_post_cont)
df.head(3)

,lication_nums,release_dates,application_dates,titles,summaries,constructions
0,M620847,2021/12/01,2021/10/18,冷水循環系統,本創作提供一種冷水循環系統，其將一冷卻裝置組包含之一第一冷卻裝置以一第一管路以及一第二管路連...,1.一種冷水循環系統，其包含： 一冷卻裝置組，其包含一第一冷卻裝置以及一第二冷卻裝置，該第一...
1,M620834,2021/12/01,2021/09/29,線路板與發光裝置CIRCUIT BOARD AND LIGHT-EMITTING APPAR...,一種線路板，包括基材、線路層、第一阻焊層以及第二阻焊層。線路層位於基材上。線路層包括第一接墊...,1.一種線路板，包括： 基材； 線路層，位於所述基材上，且所述線路層包括第一接墊及第二接墊；...
2,M620830,2021/12/01,2021/09/22,噴灑式再加熱型蒸汽反應裝置及使用其的發電設備SPRAYING AND RE-HEATING ...,一種噴灑式再加熱型蒸汽反應裝置包含：一保溫爐；一蒸汽反應爐，設置於保溫爐中；一再加熱管路，將...,1.一種噴灑式再加熱型蒸汽反應裝置，包含： 一保溫爐； 一蒸汽反應爐，設置於該保溫爐中； 一...


In [16]:
import jieba

jieba.case_sensitive = True

def do_cuts_sum_words(param_df):
    sentence = param_df["summaries"]
    words = list(jieba.cut(sentence))
    result = []
    for word in words:
        if not word or len(word) == 0 or len(word) == 1:
            continue
        word = word.lower()# 轉小寫
        result.append(word)
    return " ".join(result)

def do_cuts_con_words(param_df):
    sentence = param_df["constructions"]
    words = list(jieba.cut(sentence))
    result = []
    for word in words:
        if not word or len(word) == 0 or len(word) == 1:
            continue
        word = word.lower()# 轉小寫
        result.append(word)
    return " ".join(result)

df["sum_words"] = df.apply(do_cuts_sum_words, axis = 1)
df["con_words"] = df.apply(do_cuts_con_words, axis = 1)
# 儲存至csv
df[["lication_nums", "release_dates", "application_dates", "titles", "sum_words", "con_words"]].to_csv("wordsegs_data.csv", index = False)

df.head(5)

,lication_nums,release_dates,application_dates,titles,summaries,constructions,sum_words,con_words
0,M620847,2021/12/01,2021/10/18,冷水循環系統,本創作提供一種冷水循環系統，其將一冷卻裝置組包含之一第一冷卻裝置以一第一管路以及一第二管路連...,1.一種冷水循環系統，其包含： 一冷卻裝置組，其包含一第一冷卻裝置以及一第二冷卻裝置，該第一...,創作 提供 冷水 循環 系統 冷卻 裝置 包含 冷卻 裝置 管路 以及 管路 連通 一熱 交...,冷水 循環 系統 包含 冷卻 裝置 包含 冷卻 裝置 以及 冷卻 裝置 冷卻 裝置 設置 於...
1,M620834,2021/12/01,2021/09/29,線路板與發光裝置CIRCUIT BOARD AND LIGHT-EMITTING APPAR...,一種線路板，包括基材、線路層、第一阻焊層以及第二阻焊層。線路層位於基材上。線路層包括第一接墊...,1.一種線路板，包括： 基材； 線路層，位於所述基材上，且所述線路層包括第一接墊及第二接墊；...,線路 包括 基材 線路 阻焊 層以 及第 二阻 焊層 線路 位於 基材 線路 包括 接墊 及...,線路 包括 基材 線路 位於 所述 基材 所述 線路 包括 接墊 及第 接墊 一阻 焊層 位...
2,M620830,2021/12/01,2021/09/22,噴灑式再加熱型蒸汽反應裝置及使用其的發電設備SPRAYING AND RE-HEATING ...,一種噴灑式再加熱型蒸汽反應裝置包含：一保溫爐；一蒸汽反應爐，設置於保溫爐中；一再加熱管路，將...,1.一種噴灑式再加熱型蒸汽反應裝置，包含： 一保溫爐； 一蒸汽反應爐，設置於該保溫爐中； 一...,噴灑式 加熱型 蒸汽 反應 裝置 包含 保溫 蒸汽 反應爐 設置 保溫 爐中 一再 加熱 管...,噴灑式 加熱型 蒸汽 反應 裝置 包含 保溫 蒸汽 反應爐 設置 於該 保溫 爐中 一再 加...
3,M620822,2021/12/01,2021/09/14,集光束UVLED紫外線掃描裝置,本新型是有關於一種集光束UVLED紫外線掃描裝置，該裝置至少包括馬達軸上套裝多角形多反射面鋁...,1.一種集光束UVLED紫外線掃描裝置，至少包含：一旋轉裝置，該旋轉裝置提供一馬達，該馬達的...,新型 有關於 種集 光束 uvled 紫外線 掃描 裝置 裝置 至少 包括 馬達 軸上 套裝...,種集 光束 uvled 紫外線 掃描 裝置 至少 包含 旋轉 裝置 旋轉 裝置 提供 馬達 ...
4,M620815,2021/12/01,2021/09/14,記錄用戶足跡的系統SYSTEM FOR RECORDING USER FOOTPRINT,提供一種記錄用戶足跡的系統。所述系統包括分行端電子裝置、用戶端電子裝置及銀行端伺服器。分行端...,1.一種記錄用戶足跡的系統，包括： 一分行端電子裝置，包括一條碼擷取裝置，且設置於一地點； ...,提供 記錄 用戶 足跡 系統 所述 系統 包括 分行 電子 裝置 用戶端 電子 裝置 銀行 ...,記錄 用戶 足跡 系統 包括 分行 電子 裝置 包括 條碼 擷取 裝置 設置 地點 用戶端 ...
